<a href="https://colab.research.google.com/github/BDH-teacher/RL_from_basics/blob/main/RL_from_basic_ch_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

## 4.1 밸류 평가하기 - 반복적 정책 평가

In [ ]:
# 최적 가치 테이블 초기화

grid_0 = np.zeros((4, 4))
grid_0

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
def get_value(grid, state):
    # 상태의 row, col 값 계산
    row = state // grid.shape[1]
    col = state % grid.shape[1]

    # left, right, up, down 에 대한 밸류 계산
    left_value = 0.25 * (-1 + 1 * grid[row, max(0, col - 1)])
    right_value = 0.25 * (-1 + 1 * grid[row, min(grid.shape[1] - 1, col + 1)])
    up_value = 0.25 * (-1 + 1 * grid[max(0, row - 1), col])
    down_value = 0.25 * (-1 + 1 * grid[min(grid.shape[0] - 1, row + 1), col])

    # 밸류의 합 계산
    value = left_value + right_value + up_value + down_value

    return value

In [ ]:
# 상태 s_5에 대한 최적 가치 계산 (1행, 1열)

grid_new = np.copy(grid_0)
grid_new[1, 1] = get_value(grid_0, 5)
grid_new

array([[ 0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [ ]:
# 전체 상태 s에 대해 1회 계산 함수 정의

def update_value(grid):
    grid_new = np.copy(grid)
    for state in range(grid.shape[0] * grid.shape[1] - 1):  # 마지막 상태 15는 종료상태로 계산에서 제외
        row = state // grid.shape[1]
        col = state % grid.shape[1]
        grid_new[row, col] = get_value(grid, state)

    return grid_new

In [ ]:
# 전체 상태 s에 대해 1회 (1회 추가) 계산

grid_1 = update_value(grid_0)
grid_1

array([[-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1.,  0.]])

In [ ]:
# 전체 상태 s에 대해 2회 (1회 추가) 계산

grid_2 = update_value(grid_1)
grid_2

array([[-2.  , -2.  , -2.  , -2.  ],
       [-2.  , -2.  , -2.  , -2.  ],
       [-2.  , -2.  , -2.  , -1.75],
       [-2.  , -2.  , -1.75,  0.  ]])

In [ ]:
# 전체 상태 s에 대해 3회 (1회 추가) 계산

grid_3 = update_value(grid_2)
grid_3

array([[-3.    , -3.    , -3.    , -3.    ],
       [-3.    , -3.    , -3.    , -2.9375],
       [-3.    , -3.    , -2.875 , -2.4375],
       [-3.    , -2.9375, -2.4375,  0.    ]])

In [ ]:
# 전체 상태 s에 대해 50회 (47회 추가) 계산

grid = np.copy(grid_3)
for _ in range(47):
    grid = update_value(grid)
grid

array([[-37.13915588, -36.0222712 , -34.24567064, -32.76845942],
       [-36.0222712 , -34.37824156, -31.53427606, -28.79253866],
       [-34.24567064, -31.53427606, -26.1612582 , -19.37050043],
       [-32.76845942, -28.79253866, -19.37050043,   0.        ]])

In [ ]:
grid = np.copy(grid_3)
for _ in range(47):
    grid = update_value(grid)
grid

array([[-37.13915588, -36.0222712 , -34.24567064, -32.76845942],
       [-36.0222712 , -34.37824156, -31.53427606, -28.79253866],
       [-34.24567064, -31.53427606, -26.1612582 , -19.37050043],
       [-32.76845942, -28.79253866, -19.37050043,   0.        ]])

In [ ]:
# 전체 상태 s에 대해 200회 (150회 추가) 계산

for _ in range(150):
    grid = update_value(grid)
grid

array([[-58.32036399, -56.36427176, -53.28934376, -50.77231873],
       [-56.36427176, -53.5674438 , -48.81039425, -44.32993618],
       [-53.28934376, -48.81039425, -40.12647861, -29.47151229],
       [-50.77231873, -44.32993618, -29.47151229,   0.        ]])

In [ ]:
# 전체 상태 s에 대해 1000회 (800회 추가) 계산

for _ in range(800):
    grid = update_value(grid)
grid

array([[-59.4285713 , -57.42857131, -54.28571417, -51.71428561],
       [-57.42857131, -54.57142846, -49.71428561, -45.14285705],
       [-54.28571417, -49.71428561, -40.85714278, -29.99999994],
       [-51.71428561, -45.14285705, -29.99999994,   0.        ]])

## 4.3 최고의 정책 찾기 - 벨류 이터레이션

In [ ]:
# 최적 가치 테이블 초기화

grid_0 = np.zeros((4, 4))
grid_0

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
def get_value(grid, state):
    # 상태의 row, col 값 계산
    row = state // grid.shape[1]
    col = state % grid.shape[1]

    # left, right, up, down 에 대한 밸류 계산
    left_value = (-1 + 1 * grid[row, max(0, col - 1)])
    right_value = (-1 + 1 * grid[row, min(grid.shape[1] - 1, col + 1)])
    up_value =  (-1 + 1 * grid[max(0, row - 1), col])
    down_value = (-1 + 1 * grid[min(grid.shape[0] - 1, row + 1), col])

    # 밸류의 최댓값 계산
    value = max(left_value, right_value, up_value, down_value)

    return value

In [ ]:
# 상태 s_5에 대한 최적 가치 계산 (1행, 1열)

grid_new = np.copy(grid_0)
grid_new[1, 1] = get_value(grid_0, 5)
grid_new

array([[ 0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [ ]:
# 전체 상태 s에 대해 1회 계산 함수 정의

def update_value(grid):
    grid_new = np.copy(grid)
    for state in range(grid.shape[0] * grid.shape[1] - 1):  # 마지막 상태 15는 종료상태로 계산에서 제외
        row = state // grid.shape[1]
        col = state % grid.shape[1]
        grid_new[row, col] = get_value(grid, state)

    return grid_new

In [ ]:
# 전체 상태 s에 대해 1회 (1회 추가) 계산

grid_1 = update_value(grid_0)
grid_1

array([[-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1.,  0.]])

In [ ]:
# 전체 상태 s에 대해 2회 (1회 추가) 계산

grid_2 = update_value(grid_1)
grid_2

array([[-2., -2., -2., -2.],
       [-2., -2., -2., -2.],
       [-2., -2., -2., -1.],
       [-2., -2., -1.,  0.]])

In [ ]:
# 전체 상태 s에 대해 3회 (1회 추가) 계산

grid_3 = update_value(grid_2)
grid_3

array([[-3., -3., -3., -3.],
       [-3., -3., -3., -2.],
       [-3., -3., -2., -1.],
       [-3., -2., -1.,  0.]])

In [ ]:
# 전체 상태 s에 대해 50회 (47회 추가) 계산

grid = np.copy(grid_3)
for _ in range(47):
    grid = update_value(grid)
grid

array([[-6., -5., -4., -3.],
       [-5., -4., -3., -2.],
       [-4., -3., -2., -1.],
       [-3., -2., -1.,  0.]])

In [ ]:
# 전체 상태 s에 대해 200회 (150회 추가) 계산

for _ in range(150):
    grid = update_value(grid)
grid

array([[-6., -5., -4., -3.],
       [-5., -4., -3., -2.],
       [-4., -3., -2., -1.],
       [-3., -2., -1.,  0.]])

In [ ]:
# 전체 상태 s에 대해 1000회 (800회 추가) 계산

for _ in range(800):
    grid = update_value(grid)
grid

array([[-6., -5., -4., -3.],
       [-5., -4., -3., -2.],
       [-4., -3., -2., -1.],
       [-3., -2., -1.,  0.]])